In [ ]:
from pathlib import Path
from pyspark.sql import SparkSession

In [ ]:
def load_std_config_files(
    file_name, 
    merge_fields, 
    schema_name="config", 
    lakehouse_path="abfss://ws_tte_poc_data@onelake.dfs.fabric.microsoft.com/std_lh_tte_dominioEnagas.Lakehouse"
):
    """
    Procesa un archivo CSV dentro del mismo Lakehouse, lo guarda como tabla Delta y realiza un MERGE con una tabla existente.

    Args:
        file_name (str): Nombre del archivo CSV (sin ruta completa).
        merge_fields (list): Lista de campos con el mismo nombre en origen y destino para realizar el MERGE.
        schema_name (str): Esquema donde se guardará la tabla Delta. Por defecto, "config".
        lakehouse_path (str): Ruta base del Lakehouse. Por defecto, una ruta predefinida.
    """
    # Construir la ruta completa al archivo CSV
    csv_file_path = f"{lakehouse_path}/Files/config_schema/{file_name}"

    # Extraer el nombre del archivo sin extensión para usarlo como nombre de tabla
    table_name = f"{schema_name}.{Path(file_name).stem}"
    delta_path = f"{lakehouse_path}/Tables/{schema_name}/{Path(file_name).stem}"

    # Leer archivo CSV desde ABFSS con separador ';'
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("sep", ";") \
        .load(csv_file_path)

    # Mostrar los datos leídos
    display(df)

    # Crear tabla Delta solo si no existe en el esquema
    if not spark.catalog.tableExists(table_name):
        df.write.format("delta").mode("overwrite").save(delta_path)
        spark.sql(f"CREATE TABLE {table_name} USING DELTA LOCATION '{delta_path}'")

    # Cargar la tabla Delta existente
    from delta.tables import DeltaTable
    delta_table = DeltaTable.forPath(spark, delta_path)

    # Crear una vista temporal con los nuevos datos
    df.createOrReplaceTempView("updates")

    # Generar las condiciones de MERGE dinámicamente
    merge_conditions = " AND ".join([f"target.{field} = source.{field}" for field in merge_fields])

    # Realizar el MERGE (upsert)
    delta_table.alias("target").merge(
        df.alias("source"),
        merge_conditions
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

    # Consultar los datos de la tabla para ver los resultados
    display(spark.sql(f"SELECT * FROM {table_name}"))

In [ ]:
# Carga fichero column_mapping.csv
file_name = "column_mapping.csv"
merge_fields = ["schema_destino", "tabla_origen", "columna_origen"]

load_std_config_files(file_name, merge_fields)

In [ ]:
# Carga fichero validation_config
file_name = "validation_config.csv"
merge_fields = ["tabla_origen", "pk_columna","regla_tipo","columna_1"]

load_std_config_files(file_name, merge_fields)